In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import pprint
from sklearn.neighbors import KNeighborsClassifier
from sklearn import naive_bayes

In [16]:
data = pd.read_csv("2016_HO_ICT_stats.csv", encoding="UTF-8")
data.head(5)

,2016 ICT Losses by Home Office,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5
0,NaN,Home Office,NaN,NaN,NaN,NaN
1,Device / Incident,NaN,NaN,NaN,NaN,NaN
2,Lost Blackberries / Mobile Phones,14,13.0,27.0,19.0,73.0
3,Stolen Blackberries / Mobile Phones,2,1.0,9.0,1.0,13.0
4,Lost Laptops,2,2.0,3.0,3.0,10.0


In [17]:
data.shape

(23, 6)

In [8]:
import pandas as pd
import random
random.seed(0)

In [39]:
city = ['Moscow', 'Saint Petersburg', 'Novosibirsk','Yekaterinburg','Kazan','Chelyabinsk','Samara']
color = ['red','grey','blue','black','white'] * 5 + [None]
doors = ['2','4','4','4','5+'] * 5 + [None]
prise = ['150-200k','200-300k','300-500k','500-700k','700-1000k','1000-2000k','2000k+'] * 5 + [None]
daytime = ['morning', 'afternoon','evening','night','night'] * 5 + [None]
neighborhood = ['good','bad','bad','mixed'] * 5 + [None]
car_age = ['1','2','3','4','5+'] * 5 + [None]
found = ['found','not found', 'attempt']

In [40]:
N = 7942

data = {
    'city':[random.choice(city) for i in range(N)], 
    'color':[random.choice(color) for i in range(N)],
    'doors':[random.choice(doors) for i in range(N)],
    'prise':[random.choice(prise) for i in range(N)],
    'daytime':[random.choice(daytime) for i in range(N)],
    'neighborhood':[random.choice(neighborhood) for i in range(N)],
    'car_age':[random.choice(car_age) for i in range(N)],
    'found':[random.choice(found) for i in range(N)],
}


In [41]:
df = pd.DataFrame(data)
df.head(5)

,city,color,doors,prise,daytime,neighborhood,car_age,found
0,Chelyabinsk,grey,2,1000-2000k,evening,mixed,2,found
1,Yekaterinburg,red,4,200-300k,None,bad,4,attempt
2,Kazan,red,4,150-200k,night,mixed,3,found
3,Samara,white,2,700-1000k,evening,good,4,found
4,Moscow,red,4,2000k+,night,bad,1,not found


In [42]:
df.describe()

,city,color,doors,prise,daytime,neighborhood,car_age,found
count,7942,7637,7622,7706,7644,7575,7661,7942
unique,7,5,3,7,4,3,5,3
top,Novosibirsk,white,4,700-1000k,night,bad,2,attempt
freq,1191,1578,4449,1143,3069,3787,1569,2686


In [26]:
df.corr()

""


In [29]:
df.hist(figsize=(20, 15))

ValueError: hist method requires numerical or datetime columns, nothing to plot.

In [27]:
df.shape

(6000, 8)

In [28]:
df.to_csv('stolen-cars.csv', sep='\t', encoding='utf-8')